<a href="https://colab.research.google.com/github/yxmauw/gi-im-segmentation-2d/blob/main/Unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up environment

In [1]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!git clone https://github.com/yxmauw/gi-im-segmentation-2d.git

Cloning into 'gi-im-segmentation-2d'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 16 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (16/16), done.


In [3]:
!mkdir ~/.kaggle #Make a directory named “.kaggle”

In [4]:
!cp ./gi-im-segmentation-2d//kaggle.json ~/.kaggle/ # Copy the “kaggle.json” into this new directory

In [5]:
!chmod 600 ~/.kaggle/kaggle.json # Allocate the required permission for this file

In [6]:
!kaggle competitions download -c uw-madison-gi-tract-image-segmentation # download dataset

 99% 2.28G/2.30G [00:19<00:00, 136MB/s]
100% 2.30G/2.30G [00:19<00:00, 124MB/s]


In [ ]:
!unzip uw-madison-gi-tract-image-segmentation.zip #unzip folders

# Data

Since unable to submit for kaggle score, must generate train, validation and test set from training set

In [8]:
import numpy as np
import pandas as pd

In [14]:
# read mask annotations
mask_df = pd.read_csv('train.csv')
print(len(mask_df))

# find out how many slices have mask
im_with_mask = mask_df.loc[mask_df['segmentation'].isnull()==False, :]
print(len(im_with_mask))
display(im_with_mask.head(3))

115488
33913


,id,class,segmentation
194,case123_day20_slice_0065,stomach,28094 3 28358 7 28623 9 28889 9 29155 9 29421 ...
197,case123_day20_slice_0066,stomach,27561 8 27825 11 28090 13 28355 14 28620 15 28...
200,case123_day20_slice_0067,stomach,15323 4 15587 8 15852 10 16117 11 16383 12 166...
